In [1]:
import pandas as pd
import math
import os
pd.options.mode.chained_assignment = None

In [2]:
griddf = pd.read_csv("whgHeizTyp100m2011lowerleft00.csv")

In [3]:
griddf.head(3)

,x,y,Insg,fernw,etagen,zentral
0,4335700,2686500,3,NaN,NaN,3.0
1,4337000,2689100,3,NaN,NaN,NaN
2,4341200,2691200,3,NaN,NaN,3.0


In [4]:
griddf[["x","y"]].min()

x    4031700
y    2686500
dtype: int64

In [5]:
griddf[["x","y"]].max()

x    4671800
y    3547600
dtype: int64

In [6]:
tilingBounds= {
        "yMin": 2686500,
        "yMax": 3547600,
        "xMax": 4671800,
        "xMin": 4031700
    }

In [7]:
resolutionGeo=100

In [8]:
tileSizeCell=250

In [9]:
xIncrements = math.ceil((tilingBounds["xMax"]-tilingBounds["xMin"])/(tileSizeCell*resolutionGeo))
yIncrements = math.ceil((tilingBounds["yMax"]-tilingBounds["yMin"])/(tileSizeCell*resolutionGeo))

In [10]:
xIncrements, yIncrements

(26, 35)

In [12]:
# create numbered folders for xIncrements
for i in range(xIncrements):
    
    newpath = r'./'+str(i) 
    if not os.path.exists(newpath):
        os.makedirs(newpath)

    # filter data on x-axis
    tmpXdf = griddf[(griddf.x>=tilingBounds["xMin"]+(i*tileSizeCell*resolutionGeo)) & (griddf.x<(tilingBounds["xMin"]+(i+1)*tileSizeCell*resolutionGeo))]
        
    # iterate over the y-axis
    for j in range(yIncrements):
        
        # filter data to tile
        tmpXYdf = tmpXdf[(tmpXdf.y>=tilingBounds["yMin"]+(j*tileSizeCell*resolutionGeo)) & (tmpXdf.y<(tilingBounds["yMin"]+(j+1)*tileSizeCell*resolutionGeo))]

        # reduce grid coordinates to reference tile instead of originPoint
        tmpXYdf["x"] = tmpXYdf["x"].apply(lambda x: (x-tilingBounds["xMin"]-i*tileSizeCell*resolutionGeo)/resolutionGeo)
        tmpXYdf["y"] = tmpXYdf["y"].apply(lambda x: (x-tilingBounds["yMin"]-j*tileSizeCell*resolutionGeo)/resolutionGeo)

        # write to csv if tile contains cells
        if len(tmpXYdf)>0:
            
            tmpXYdf.to_csv(f'./{i}/{j}.csv', float_format='%.0f', index=False)